Project: From Insight → Action

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('advertising.csv')

In [ ]:
df['ctr'] = (df['clicks']/df['impressions']).round(4)
df['cpc'] = (df['cost']/df['clicks']).round(2)

df.head()

,ad_id,campaign,clicks,impressions,cost,ctr,cpc
0,1,Summer_Sale,120,5000,250,0.0240,2.08
1,2,Summer_Sale,95,4200,210,0.0226,2.21
2,3,Summer_Sale,60,3000,180,0.0200,3.00
3,4,App_Launch,200,8000,500,0.0250,2.50
4,5,App_Launch,180,7600,480,0.0237,2.67


In [26]:
# campaign | ctr | cpc | spend_share
# summary_df = df.groupby('campaign')[['ctr','cpc','cost']].agg(
#     avg_ctr = ('ctr','mean'),
#     avg_cpc = ('cpc','mean'),
#     sum_cost = ('cost','sum')
# ).reset_index()
# summary_df['spend_share'] = summary_df['sum_cost'] /df['cost'].sum()
# summary_df


# CTR and CPC must be recomputed from summed metrics, not averaged — averaging ratios is a rookie mistake.
summary_df = (
    df.groupby('campaign',as_index=False).agg(
        clicks=('clicks','sum'),
        impressions = ('impressions','sum'),
        cost = ('cost','sum')
    )
)
summary_df['ctr'] = summary_df['clicks'] / summary_df['impressions']
summary_df['cpc'] = summary_df['cost'] / summary_df['clicks']
summary_df['spend_share'] = (summary_df['cost'] / summary_df['cost'].sum()) *100
summary_df[['campaign','ctr','cpc','spend_share']]

,campaign,ctr,cpc,spend_share
0,App_Launch,0.022816,2.723404,32.323232
1,Brand_Awareness,0.006190,5.153846,16.919192
2,Festive_Offer,0.031915,1.826667,34.595960
3,Summer_Sale,0.022541,2.327273,16.161616


Answer 3 EXECUTIVE QUESTIONS:

1. Which campaign should we scale?

2. Which campaign should be paused?

3. What 1 change would improve ROI fastest?

In [59]:
summary_df['efficiency_score'] = summary_df['ctr'] / summary_df['cpc']

scale = summary_df.sort_values(
    by=['efficiency_score', 'spend_share'],
    ascending=[False, False]
).head(1)

pause = summary_df.sort_values(
    by=['efficiency_score', 'spend_share'],
    ascending=[True, False]
).head(1)

scale_campaign=scale['campaign'].iloc[0]
pause_campaign = pause['campaign'].iloc[0]


In [68]:
# Which campaign should we scale?
print(f'\nWe should Scale {scale_campaign} because it has high ctr and low cpc')
# Which campaign should be paused?
print(f'\nWe should pause {pause_campaign} because it has low ctr and high cpc\n')
# What 1 change would improve ROI fastest?
print(scale[['campaign','efficiency_score']] ,'\n',pause[['campaign','efficiency_score']],f'\nThese are the best and worst campaigns. Invest more in {scale_campaign} and pause {pause_campaign} would improve ROI')


We should Scale Festive_Offer because it has high ctr and low cpc

We should pause Brand_Awareness because it has low ctr and high cpc

        campaign  efficiency_score
2  Festive_Offer          0.017472 
           campaign  efficiency_score
1  Brand_Awareness          0.001201 
These are the best and worst campaigns. Invest more in Festive_Offer and pause Brand_Awareness would improve ROI


In [70]:
'''
    1. Scale Festive_Offer because it has high ctr and low cpc with high spending already
    2. Pause Brand_Awareness because it has low ctr and high cpc with low spending currently
    3. Move the budget of paused campaign to scale campaign
'''

'\n    1. Scale Festive_Offer because it has high ctr and low cpc with high spending already\n    2. Pause Brand_Awareness because it has low ctr and high cpc with low spending currently\n    3. Move the budget of paused campaign to scale campaign\n'

In [71]:
summary_df.to_csv('summary_df.csv')